<a href="https://www.kaggle.com/code/mervesenakeskin/hybrid-recommender-system-1?scriptVersionId=113225039" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **User-Based Collaborative Filtering ( Kullanıcı Tabanlı İşbirlikçi Filtreleme )**



* Bir arkadaş gurubu olduğunu düşünelim. Bu arkadaş gurubunun izlemiş olduğu filmlere benzer puanlar verdiğini varsayalım. Bu guruptakilerin izleyip beğendiği filmi, grupta izlemedeği bilinen kişiye bu filmin önerisini yapılırsa benzer davranış sergileyecektir.Burdan yola çıkarak kullanıcı benzerliği tabanlı filtreleme yapacağız. 
***Adımlar**
* Verinin Hazırlanması
* Kullanıcı seçilmesi
* Benzer davranış gösteren bireylerin ID lerinin bulunması
* En Benzer Davranışlı Kullanıcıların Belirlenmesi
* Weighted Average Recommendation Score'un Hesaplanması


In [ ]:
from IPython.display import Image
import os
!ls ../input/recommendersystemimage/1.png
Image("../input/png1ckdmvsdfv/User BasedRecommendation.png")

* AMAÇ = ID'si verilen kullanıcı için item-based ve user based recommender yöntemlerini kullanarak 10 film önerisi yapınız.
* Veris Seti : 27.278 film 2.000.0263 derecelendirme içermektedir.

Dataset Açıklama:

**movie.csv**

* movieId : Eşsiz film numarası
* title   : Film Adı
* genres  : Filmin Türü 


**rating.csv**

* userid    : Eşsiz kullanıcı numarası
* movieId   : Eşsiz film numarası
* rating    : Kullanıcı tarafından filme verilen puan
* timestamp : Değerlendirme tarihi


**1. Verinin Hazırlanması**

In [ ]:
import pandas as pd
import seaborn as sns
import random

movie = pd.read_csv("../input/movie-lens-dataset/movies.csv")
rating = pd.read_csv("../input/movie-lens-dataset/ratings.csv")

#Veri setlerinin birleştirilmesi
df = movie.merge(rating, how="left", on="movieId")

# 10000 den az oya sahip olan filmlerin DataFrame den çıkartılması
comment_counts = pd.DataFrame(df["title"].value_counts())
common_movies_ = comment_counts[comment_counts["title"] <= 10000].index
common_movies = df[~df["title"].isin(common_movies_)]

# Pivotunu alarak veriyi işlenecek hale çevrilmesi
user_movie_df = df.pivot_table(index=["userId"],columns=["title"],values="rating")

In [ ]:
 user_movie_df.head(10)

**2 Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi**

In [ ]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=250).values)

random_user_df = user_movie_df[user_movie_df.index == random_user]

movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()



In [ ]:
# rastgele seçtiğimiz kullanıcının izlediği filmler
movies_watched

**3 Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişilmesi**

In [ ]:
# user_movie_df içerisinden random_user ın izledği filmleri yeni bir dataFrame atılır ve burdan null değerler çıakrtılır.
movies_watched_df = user_movie_df[movies_watched]
user_movie_count = movies_watched_df.T.notnull().sum()

user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]

# random_user ile %60 oranında aynı izlemiş olan kullanıcıların ıd lerine erişilir. 
users_same_movies = user_movie_count[user_movie_count["movie_count"] > len(movies_watched)*0.6]["userId"]


In [ ]:
user_movie_count

**4 Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi**

In [ ]:
#  user_same_movies listesi içerisindeki seçili userile benzerlik gösteren kullanıcıların id’lerinin bulunacağı şekilde movies_watched_dfdataframe’ini filtreleyiniz

movies_watched_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

#Kullanıcıların birbirleri ile olan korelasyonlarının bulunması

corr_df = movies_watched_df.T.corr().unstack().sort_values().drop_duplicates()

# kolay okumak için:

corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()

# yüksek korelasyona sahip kullanıcıları filtreleyerek yeni bir dataFrame alalım

top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.4)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

# bu kullanıcları rating  setindeki tüm verilerini getirelim ve random_user ın kendisini bbu veriden cıkartalım 

top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]




In [ ]:
corr_df

In [ ]:
top_users

**5 Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması**

In [ ]:
# bulduğumuz kullanıcların scorenun hesaplabilmek için ağırlıklandırılmış yeni bir dataFrame oluşturalım
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

# sıralamak için:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
user_movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 2.5].sort_values("weighted_rating", ascending=False)

# öneri aşaması:
user_movies_to_be_recommend = user_movies_to_be_recommend.merge(movie[["movieId", "title"]]).head()
# Kullanıcının daha önce izlemediği filmleri çıkaralım
user_movies_to_be_recommend = user_movies_to_be_recommend.loc[~user_movies_to_be_recommend["title"].isin(movies_watched)][:5]


In [ ]:
user_movies_to_be_recommend